In [4]:
import pandas as pd
import numpy as np
import holidays
from pathlib import Path
from sklearn.preprocessing import FunctionTransformer, OneHotEncoder, StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LinearRegression, Ridge
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV
from flaml import AutoML
from xgboost import XGBRegressor
from skrub import TableVectorizer, DatetimeEncoder
import optuna
import importlib
import bike_count as bc
import xgboost as xgb
from sklearn.metrics import root_mean_squared_error

In [ ]:
def xgb_vectorized_for_optuna(trial=None):

    learning_rate = trial.suggest_float("learning_rate", 0.01, 0.3) if trial else 0.1
    max_depth = trial.suggest_int("max_depth", 3, 15) if trial else 10
    n_estimators = trial.suggest_int("n_estimators", 50, 200) if trial else 100

    regressor = XGBRegressor(
        learning_rate=learning_rate,
        n_estimators=n_estimators,
        max_depth=max_depth,
        random_state=42,
        tree_method='hist',
        enable_categorical=True,
    )

    pipe = make_pipeline(regressor)
    
    return pipe

In [5]:
importlib.reload(bc)

<module 'bike_count' from 'c:\\Users\\diane\\bike_count_ldmh\\bike_count.py'>

In [9]:
X, y = bc.get_model_data()

date = bc.date_encoder
table = bc.table_vectorizer
merg = bc.merge

X = merg.fit_transform(X)
X = table.fit_transform(X)

X_train, X_val, y_train, y_val = bc.train_test_temporal(X, y)

X_train = date.fit_transform(X_train)
X_val = date.fit_transform(X_val)

KeyError: 'date'

In [7]:
def objective(trial):
    param = {
        "objective": "reg:squarederror",  # ou 'binary:logistic' pour un problème de classification binaire
        "eval_metric": "rmse",  # Pour un problème de régression, ou 'logloss' pour classification
        "max_depth": trial.suggest_int("max_depth", 8, 15),
        "min_child_weight": trial.suggest_int("min_child_weight", 1, 10),
        "subsample": trial.suggest_float("subsample", 0.5, 1.0),
        "colsample_bytree": trial.suggest_float("colsample_bytree", 0.5, 1.0),
        "alpha": trial.suggest_float(
            "alpha", 1e-4, 1e-1, log=True
        ),  # L2 regularization term
        "lambda": trial.suggest_float("lambda", 1e-4, 1e-1, log=True),
        "n_estimators": trial.suggest_float(
            "n_estimators", 50, 150
        ),  # L1 regularization term
    }

    model = xgb.XGBRegressor(**param)
    model.fit(X_train, y_train, eval_set=[(X_val, y_val)], verbose=False)

    y_pred = model.predict(X_val)
    rmse = root_mean_squared_error(
        y_val, y_pred
    )  # ou accuracy_score pour classification

    return rmse

In [8]:
study = optuna.create_study(direction="minimize")
study.optimize(objective, n_trials=200)

print("Meilleurs paramètres : ", study.best_params)
print("Meilleure métrique (RMSE) : ", study.best_value)

[I 2024-12-11 17:47:32,618] A new study created in memory with name: no-name-0df5886e-ffdb-446a-b8c4-b9cf0b6ab1b0
[W 2024-12-11 17:47:35,686] Trial 0 failed with parameters: {'max_depth': 13, 'min_child_weight': 2, 'subsample': 0.992601613350111, 'colsample_bytree': 0.9138727331980381, 'alpha': 0.0016369844657114457, 'lambda': 0.00014658034383620126, 'n_estimators': 111.63797643786754} because of the following error: ValueError('feature_names mismatch: [\'counter_id: 100057445, 104057445, 103057445\', \'counter_id: 100056329, 103056329, 104056329\', \'counter_id: 102007049, 101007049, 100007049\', \'counter_id: 100047542, 104047542, 103047542\', \'counter_id: 100044493, sc, 100049407\', \'counter_id: 104036719, 100036719, 103036719\', \'counter_id: 100056330, 103056330, 104056330\', \'counter_id: 102060178, 101060178, 100060178\', \'counter_id: 100057329, 104057329, 103057329\', \'counter_id: 100056046, 100056047, sc\', \'counter_id: 353277235, 353277233, 100063175\', \'counter_id: 100

ValueError: feature_names mismatch: ['counter_id: 100057445, 104057445, 103057445', 'counter_id: 100056329, 103056329, 104056329', 'counter_id: 102007049, 101007049, 100007049', 'counter_id: 100047542, 104047542, 103047542', 'counter_id: 100044493, sc, 100049407', 'counter_id: 104036719, 100036719, 103036719', 'counter_id: 100056330, 103056330, 104056330', 'counter_id: 102060178, 101060178, 100060178', 'counter_id: 100057329, 104057329, 103057329', 'counter_id: 100056046, 100056047, sc', 'counter_id: 353277235, 353277233, 100063175', 'counter_id: 100050876, 104050876, 103050876', 'counter_id: 100057380, 104057380, 103057380', 'counter_id: 100047547, 103047547, 103047546', 'counter_id: 100056226, 104056226, 103056226', 'counter_id: 109042374, 110042374, 100042374', 'counter_id: 100056331, 103056331, 104056331', 'counter_id: 104036718, 100036718, 103036718', 'counter_id: 105056336, 106056336, 100056336', 'counter_id: 104056332, 104056330, 104056335', 'counter_id: 100056327, 103056327, 10

In [63]:
X, y = bc.get_model_data()
pipe = bc.xgb_vectorized_no_date_encoding()

pipe.fit(X, y)

Pipeline(steps=[('functiontransformer-1',
                 FunctionTransformer(func=<function _merge_external_data at 0x000001CB894AE320>)),
                ('functiontransformer-2',
                 FunctionTransformer(func=<function _encode_date at 0x000001CB894AD1B0>)),
                ('tablevectorizer',
                 TableVectorizer(cardinality_threshold=100,
                                 datetime=DatetimeEncoder(add_total_seconds=False,
                                                          resolution='month'),
                                 n_jobs...
                              feature_types=None, gamma=None, grow_policy=None,
                              importance_type=None,
                              interaction_constraints=None, learning_rate=None,
                              max_bin=None, max_cat_threshold=None,
                              max_cat_to_onehot=None, max_delta_step=None,
                              max_depth=10, max_leaves=None, min_child_weight=7,
                              missing=nan, monotone_constraints=None,
                              multi_strategy=None, n_estimators=None,
                              n_jobs=None, num_parallel_tree=None,
                              random_state=None, ...))])

In [64]:
test_data = pd.read_parquet("data/final_test.parquet")
test_pred = pipe.predict(test_data)

test_df = pd.DataFrame({"Id": range(len(test_pred)), "log_bike_count": test_pred})
test_df.to_csv("opti_predictions.csv", index=False)

In [5]:
study = optuna.create_study(direction="minimize")
study.optimize(bc.objective, n_trials=50)

print("Meilleurs paramètres : ", study.best_params)
print("Meilleure métrique (RMSE) : ", study.best_value)

[I 2024-12-11 12:16:49,541] A new study created in memory with name: no-name-e10a81f1-b967-433e-852b-f4b242f23465
[I 2024-12-11 12:18:19,616] Trial 0 finished with value: 0.49843714390187704 and parameters: {'learning_rate': 0.1796755184532604, 'max_depth': 8, 'n_estimators': 111}. Best is trial 0 with value: 0.49843714390187704.
[I 2024-12-11 12:19:30,324] Trial 1 finished with value: 0.690716987124988 and parameters: {'learning_rate': 0.01835105263985338, 'max_depth': 6, 'n_estimators': 122}. Best is trial 0 with value: 0.49843714390187704.
[I 2024-12-11 12:20:45,988] Trial 2 finished with value: 0.44309938739833055 and parameters: {'learning_rate': 0.05830528906289511, 'max_depth': 14, 'n_estimators': 63}. Best is trial 2 with value: 0.44309938739833055.
[I 2024-12-11 12:22:52,973] Trial 3 finished with value: 0.5182766825295525 and parameters: {'learning_rate': 0.1718320626492706, 'max_depth': 4, 'n_estimators': 148}. Best is trial 2 with value: 0.44309938739833055.
[I 2024-12-11 1

Meilleurs paramètres :  {'learning_rate': 0.16878974156327872, 'max_depth': 11, 'n_estimators': 139}
Meilleure métrique (RMSE) :  0.42235674625423486


### Predictions

In [24]:
importlib.reload(bc)

<module 'bike_count' from 'c:\\Users\\diane\\bike_count_ldmh\\bike_count.py'>

In [25]:
X, y = bc.get_model_data()

pipe = bc.xgb_vectorized_no_date_encoding()
pipe.fit(X, y)

test_data = pd.read_parquet("data/final_test.parquet")
test_pred = pipe.predict(test_data)

test_df = pd.DataFrame({"Id": range(len(test_pred)), "log_bike_count": test_pred})
test_df.to_csv("optimized_predictions.csv", index=False)

KeyboardInterrupt: 

In [10]:
importlib.reload(bc)

X, y = bc.get_model_data()
pipe = bc.rf_vectorized_no_date_encoding()

pipe.fit(X, y)

Pipeline(steps=[('functiontransformer-1',
                 FunctionTransformer(func=<function _merge_external_data at 0x000001A43265BA30>)),
                ('functiontransformer-2',
                 FunctionTransformer(func=<function _encode_date at 0x000001A40AFE7BE0>)),
                ('functiontransformer-3',
                 FunctionTransformer(func=<function drop_columns at 0x000001A4352B2680>)),
                ('tablevectorizer',
                 TableVectorizer(datetime=DatetimeEncoder(add_total_seconds=False,
                                                          resolution='month'),
                                 n_jobs=-1)),
                ('randomforestregressor',
                 RandomForestRegressor(max_depth=10, random_state=42))])

In [11]:
test_data = pd.read_parquet("data/final_test.parquet")
test_pred = pipe.predict(test_data)

test_df = pd.DataFrame({"Id": range(len(test_pred)), "log_bike_count": test_pred})
test_df.to_csv("optimized_predictions.csv", index=False)